In [11]:
from pyspark.sql import SparkSession

In [12]:
from pyspark.sql.functions import explode,col,expr,array,when
from pyspark.sql.types import ArrayType, IntegerType, ShortType

In [13]:
spark = SparkSession.builder.appName("task").getOrCreate()

In [14]:
spark

In [15]:
df_pyspark = spark.read.option('multiline','true').json('provider.json')

In [16]:
df_pyspark.printSchema()

root
 |-- provider_group_id: long (nullable = true)
 |-- provider_groups: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- npi: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- tin: struct (nullable = true)
 |    |    |    |-- type: string (nullable = true)
 |    |    |    |-- value: string (nullable = true)



In [17]:
provider_group = df_pyspark.withColumn("new_provider", explode("provider_groups"))
provider_new = provider_group.withColumn("npi",explode("new_provider.npi"))
provider_again = provider_new.select(
    "provider_group_id",
    col("npi").alias("npi"),
    col("new_provider.tin.type").alias("tin_type"),
    col("new_provider.tin.value").alias("tin")
)

In [18]:
provider_again.printSchema()

root
 |-- provider_group_id: long (nullable = true)
 |-- npi: long (nullable = true)
 |-- tin_type: string (nullable = true)
 |-- tin: string (nullable = true)



In [19]:
provider_again.show()

+-----------------+----------+--------+----------+
|provider_group_id|       npi|tin_type|       tin|
+-----------------+----------+--------+----------+
|         10001001|1235233008|     ein|04-3267217|
|         10001001|1316041189|     ein|04-3267217|
|         10001001|1780788554|     ein|04-3267217|
|         10001001|1891068409|     ein|04-3267217|
|         10001001|1366459570|     ein|11-1562701|
|         10001001|1417915653|     ein|11-3358535|
|         10001001|1417915653|     ein|13-3888838|
|         10002001|1609829761|     ein|00-0004110|
|         10002001|1821482241|     ein|00-0004110|
|         10002001|1760986277|     ein|00-6980743|
|         10002001|1215075882|     ein|01-0550744|
|         10002001|1013917665|     ein|01-0555304|
|         10002001|1679780811|     ein|01-0555483|
|         10002001|1700093952|     ein|01-0555483|
|         10002001|1780072447|     ein|01-0555483|
|         10002001|1952532970|     ein|01-0555483|
|         10002001|1376647511| 

In [20]:
provider_cast = provider_again.withColumn("tin_type", col("tin_type").cast(ShortType()))

In [21]:
provider_cast.printSchema()

root
 |-- provider_group_id: long (nullable = true)
 |-- npi: long (nullable = true)
 |-- tin_type: short (nullable = true)
 |-- tin: string (nullable = true)



In [22]:
provider_cast.write.parquet('provider_output.parquet')